In [1]:
import sys
sys.path.append('../harmonizome')

import pandas as pd
import numpy as np
import utility_functions as uf
import timeit
import itertools
import scipy.spatial.distance as dist

In [2]:
%load_ext autoreload
%load_ext memory_profiler
%autoreload 2

### removeAndInpute

In [3]:
df = pd.DataFrame(data=np.random.randint(10,size=(5, 10)))
df = df.mask(np.random.random(df.shape) < .1)
df.head()

,0,1,2,3,4,5,6,7,8,9
0,2,1,7.0,3,6,NaN,4.0,2,8.0,3.0
1,6,8,5.0,0,4,3.0,NaN,5,8.0,NaN
2,5,8,2.0,7,2,7.0,1.0,0,6.0,6.0
3,3,0,NaN,2,2,2.0,5.0,1,2.0,7.0
4,9,8,5.0,5,3,NaN,4.0,3,NaN,8.0


In [4]:
df.shape

(5, 10)

## makeBinaryMatrix

In [31]:
df = pd.DataFrame(data=np.random.randint(3000,size=(7500, 2)), columns=['Gene Symbol', 'Attributes'])
df = df.set_index('Gene Symbol')
df.head()

,Attributes
Gene Symbol,
1460,2085
2012,395
958,2260
421,2326
1230,2555


In [32]:
%mprun -f uf.createBinaryMatrix binary_matrix = uf.createBinaryMatrix(df)

Filename: ../harmonizome/utility_functions.py

Line #    Mem usage    Increment   Line Contents
   404    883.2 MiB    883.2 MiB   def createBinaryMatrix(df):
   405                                 '''
   406                                 Creates an adjacency matrix from df, which is a gene-attribute edge
   407                                 list.
   408                                 '''
   409    883.2 MiB      0.0 MiB       matrix = pd.crosstab(df.index, df.iloc[:, 0]) > 0
   410    883.2 MiB      0.0 MiB       matrix.index.name = df.index.name
   411    883.2 MiB      0.0 MiB       matrix.columns.name = df.columns[0]
   412    883.2 MiB      0.0 MiB       return matrix

In [33]:
%%memit
binary_matrix.shape

peak memory: 883.25 MiB, increment: 0.00 MiB


In [41]:
%mprun -f uf.createSimilarityMatrix similarityuf = uf.createSimilarityMatrix(binary_matrix, 'jaccard', sparse=True)

Filename: ../harmonizome/utility_functions.py

Line #    Mem usage    Increment   Line Contents
   202    883.4 MiB    883.4 MiB   def createSimilarityMatrix(inputDF, metric, dtype=None, sparse=False):
   203                                 '''
   204                                 Creates a similarity matrix between the rows of the inputDF based on
   205                                 the metric specified. The resulting matrix has both rows and columns labeled
   206                                 by the index of inputDF.
   207                                 '''
   208    883.4 MiB      0.0 MiB       if sparse and metric == 'jaccard':
   209                                     # from na-o-ys on Github
   210    883.4 MiB      0.0 MiB           sparse = sp.csr_matrix(inputDF.to_numpy(dtype=np.bool).astype(np.int))
   211    883.4 MiB      0.0 MiB           cols_sum = sparse.getnnz(axis=1)
   212    883.4 MiB      0.0 MiB           ab = sparse * sparse.T
   213    883.4 MiB      0

In [35]:
%mprun -f uf.createSimilarityMatrix similarity_matrix = uf.createSimilarityMatrix(binary_matrix, 'jaccard')

Filename: ../harmonizome/utility_functions.py

Line #    Mem usage    Increment   Line Contents
   202    883.2 MiB    883.2 MiB   def createSimilarityMatrix(inputDF, metric, dtype=None, sparse=False):
   203                                 '''
   204                                 Creates a similarity matrix between the rows of the inputDF based on
   205                                 the metric specified. The resulting matrix has both rows and columns labeled
   206                                 by the index of inputDF.
   207                                 '''
   208    883.2 MiB      0.0 MiB       if sparse and metric == 'jaccard':
   209                                     # from na-o-ys on Github
   210                                     sparse = sp.csr_matrix(inputDF.to_numpy(dtype=np.bool).astype(np.int))
   211                                     cols_sum = sparse.getnnz(axis=1)
   212                                     ab = sparse * sparse.T
   213                    

In [42]:
np.allclose(similarityuf, similarity_matrix)

True